In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras import regularizers
import random
import subprocess as sp
import os
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [2]:
model = load_model('saved_models/best_model2.h5') 
train_stats = pd.read_pickle('saved_models/mode2_stat.pkl')

In [3]:
def norm(x):
    return (x - train_stats['mean']) / (train_stats['std']+0.0000001)

def log_norm(x):
    for col in x.columns:
        x.iloc[:,col] = (x.iloc[:,col] - x.iloc[:,col].min()+1).transform(np.log10)
    return x

In [5]:
# verifying the model with trained data
X_train = pd.read_csv('dataset/updated_model_dataset/Final_dataset/vopd_dataset.csv',header=None,error_bad_lines=False)
Y_train = X_train.iloc[:,-1]
X_train.drop(X_train.columns[-1],inplace=True,axis=1)
# X_train.drop(X_train.columns[-1],inplace=True,axis=1)
X = norm(X_train)
X.drop(X.columns[0],inplace=True,axis=1)
Y_pred = model.predict(X).flatten()
train_MSE = np.sum(np.square(Y_pred-Y_train))/Y_train.shape[0]
print('train MSE = %f'%(train_MSE))

#plot histogram
train_error = Y_pred-Y_train
plt.hist(train_error, bins =25,alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel("Prediction Error")
plt.ylabel("Count")
plt.title('train_data_hist')
plt.savefig('train_histr.jpg')

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [5]:
def mapp_syntraff(traffic_type,inject_rate):
#     random.seed(25)
         
    file2 = open("latency_model_2010_pso/files/vopd_mapp.txt","w")
    file1 = open('booksim/examples/vopd_mapp.txt','w')
    for i in range(1000):
        mapp_vec = random.sample(range(1,17),16)
        file2.write('%d\t %s \t %f\n'%(i,str(mapp_vec),inject_rate))
        file1.write('%d\t %s \t %f\n'%(i,str(mapp_vec),inject_rate))
    file2.close()
    file1.close()
    
    # ************* begin modify file  files/config.txt for diff syn traffic pattern *************#
    with open('latency_model_2010_pso/files/config.txt', 'r') as file:
        # read a list of lines into data
        data = file.readlines()


    # now change the 2nd line, note that you have to add a newline
    data[3] = 'traffic file#\tfiles/'+traffic_type+'\n'

    # and write everything back
    with open('latency_model_2010_pso/files/config.txt', 'w') as file:
        file.writelines( data )
    #************** end modifying the file **************************************************#
    open('latency_model_2010_pso/files/vopd_mapp_features.txt','w').close()
    os.chdir('/home/eslab_server/Desktop/Ramesh/queuing_work/mesh4x4_training/latency_model_2010_pso/')
    os.system('./latency_model_2010_pso')
    os.chdir('/home/eslab_server/Desktop/Ramesh/queuing_work/mesh4x4_training/')

    with open('latency_model_2010_pso/files/vopd_mapp_features.txt','r') as my_file:
        text = my_file.read()
        text = text.replace("    ", " ")
        text = text.replace("[", " ")
        text = text.replace("]", " ")
        text = text.replace(" ",",")
        text = text.replace(",,,",",")
        text = text.replace(",,",",")
        text = text.replace(",,",",")
        text = text.replace(",\n,\n,\n,",'\n')
        text = text.replace('\n,',',')
    my_file.close()
    
    open('latency_model_2010_pso/files/vopd_mapp_features.csv', 'w').close()
    with open('latency_model_2010_pso/files/vopd_mapp_features.csv','w') as analy_file:
        analy_file.write(text)


    # Latency evaluation using booksim simulator
    os.chdir('/home/eslab_server/Desktop/Ramesh/queuing_work/mesh4x4_training/booksim/')
#     print(os.getcwd())
        # ************* begin modify file  files/config.txt for diff syn traffic pattern *************#
    with open('examples/anynet_config', 'r') as file:
        # read a list of lines into data
        trff = file.readlines()


    # now change the 2nd line, note that you have to add a newline
    trff[73] = 'traffic_file = examples/'+traffic_type+';\n'

    # and write everything back
    with open('examples/anynet_config', 'w') as file:
        file.writelines(trff)
    #************** end modifying the file **************************************************#
    
    open('examples/vopd_mapp_out.txt', 'w').close()
    sp.call('./pgm_within_pgm',shell=True)

    with open('examples/vopd_mapp_out.txt','r') as my_file:
        text = my_file.read()
        text = text.replace(" \t ", ",")
        text = text.replace("\t\t", ",")
        text = text.replace("\t", ",")
        text = text.replace("[", "")
        text = text.replace("]", " ")
        text = text.replace(" ","")
    my_file.close()
    open('/home/eslab_server/Desktop/Ramesh/queuing_work/mesh4x4_training/booksim_out.csv', 'w').close()
    with open('/home/eslab_server/Desktop/Ramesh/queuing_work/mesh4x4_training/booksim_out.csv','w') as myfile:
        myfile.write(text)
    my_file.close()
#     print(os.getcwd())
    os.chdir('/home/eslab_server/Desktop/Ramesh/queuing_work/mesh4x4_training')
#     print(os.getcwd())

    X = pd.read_csv("latency_model_2010_pso/files/vopd_mapp_features.csv",header=None,
                          error_bad_lines=False)
    book_lat = pd.read_csv('booksim_out.csv',header=None,
                          error_bad_lines=False)
    X.drop(X.columns[-1],inplace=True,axis=1)
    anay_lat = X.iloc[:,-1]
    
    
    
    X_norm = norm(X)
    X_norm.drop(X_norm.columns[0],inplace=True,axis=1)
    Y = model.predict(X_norm).flatten()

    return anay_lat,Y,book_lat

In [20]:
os.chdir('/home/eslab_server/Desktop/Ramesh/queuing_work/mesh4x4_training/')

# trff_files = ['traffic_bitcomplement_16.txt','traffic_bitreverse_16.txt',
#               'traffic_neighbor_16.txt','traffic_shuffle_16.txt','traffic_tornado_16.txt',
#               'traffic_transpose_16.txt','traffic_uniform_16.txt','traffic_vopd.txt','traffic_vopd.txt']

trff_files = ['traffic_vopd.txt','traffic_vopd.txt','traffic_vopd.txt','traffic_vopd.txt',
              'traffic_vopd.txt','traffic_vopd.txt','traffic_vopd.txt','traffic_vopd.txt',
              'traffic_vopd.txt']

anay_lat,Y,book_lat = mapp_syntraff(trff_files[0],0.007)
   



In [21]:
#  Relative error between analy model and Booksim
err_anal = np.abs(book_lat.iloc[:,18]-anay_lat)
rel_err = np.sum(err_anal/book_lat.iloc[:,18])/len(err_anal)
print('Relative error b/w booksim and analy model=%f'%rel_err)

Relative error b/w booksim and analy model=0.038216


In [22]:
#  Relative error between analy model and Booksim
err_DNN = np.abs(book_lat.iloc[:,18]-Y)
rel_err = np.sum(err_DNN/book_lat.iloc[:,18])/len(err_DNN)
print('Relative error b/w booksim and DNN model=%f'%rel_err)

Relative error b/w booksim and DNN model=0.029736


In [16]:
 book_lat = pd.read_csv('booksim_out.csv',header=None,error_bad_lines=False)
err_anal = np.zeros(1000)
rel_err = np.zeros(1000)
exp = [677]
for i in range(len(anay_lat)):
    if i not in exp:
        err_anal[i] = np.abs(book_lat.iloc[:,18][i]-anay_lat[i])
        rel_err[i]  = err_anal[i]/book_lat.iloc[:,18][i]
print('Relative error b/w booksim and analy model=',np.sum(rel_err)/len(rel_err))

Relative error b/w booksim and analy model= 0.1441213550301619
